<a href="https://colab.research.google.com/github/MineManiac/superComputacao-2024.2/blob/main/Aula11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
%%writefile exercicio.cu
#include <thrust/device_vector.h>   // Inclui a biblioteca Thrust para vetores na GPU
#include <thrust/host_vector.h>     // Inclui a biblioteca Thrust para vetores na CPU
#include <thrust/extrema.h>         // Inclui a biblioteca para encontrar mínimo e máximo
#include <iostream>                 // Biblioteca padrão para entrada e saída de dados
#include <chrono>                   // Biblioteca para medir o tempo de execução do código
using namespace std;

int main() {
    int n = 2518;                   // Define o tamanho do vetor (2518 elementos)
    double value = 0.0;             // Variável auxiliar para armazenar o valor lido da entrada
    std::chrono::duration<double> diff; // Variável para armazenar a diferença de tempo calculada

    // Marca o início da medição do tempo de leitura dos dados
    auto leitura_i = std::chrono::steady_clock::now();

    // Cria um vetor na CPU (host_vector) com n elementos, todos inicializados com 0
    thrust::host_vector<double> host(n, 0);

    // Loop para ler n valores da entrada padrão e armazená-los no vetor host
    for (int i = 0; i < n; i++) {
        cin >> value;              // Lê um valor da entrada padrão
        host[i] = value;           // Armazena o valor lido no vetor host
    }

    // Marca o fim da medição do tempo de leitura dos dados
    auto leitura_f = std::chrono::steady_clock::now();

    // Calcula o tempo gasto na leitura dos dados
    diff = leitura_f - leitura_i;

    // Exibe o tempo de leitura em segundos
    cout << "Tempo de LEITURA (em segundos)  " << diff.count() << endl;

    // Marca o início da medição do tempo de cópia dos dados da CPU para a GPU
    auto copia_i = std::chrono::steady_clock::now();

    // Cria um vetor na GPU (device_vector) copiando os dados do vetor host da CPU
    thrust::device_vector<double> dev(host);

    // Marca o fim da medição do tempo de cópia dos dados
    auto copia_f = std::chrono::steady_clock::now();

    // Calcula o tempo gasto na cópia dos dados para a GPU
    diff = copia_f - copia_i;

    // Exibe o tempo de cópia em segundos
    cout << "Tempo de CÓPIA (em segundos)  " << diff.count() << endl;

    // Calcula a soma e a média dos elementos
    double soma = thrust::reduce(dev.begin(), dev.end(), 0.0, thrust::plus<double>());
    std::cout << "Soma dos elementos: " << soma << std::endl;
    double media = soma / dev.size();
    std::cout << "Média dos elementos: " << media << std::endl;

    // Encontrar o menor e maior preço do período total
    auto total_minmax = thrust::minmax_element(dev.begin(), dev.end());
    double total_min = *total_minmax.first;
    double total_max = *total_minmax.second;
    std::cout << "Menor preço do período total: " << total_min << std::endl;
    std::cout << "Maior preço do período total: " << total_max << std::endl;


    return 0;
}


Overwriting exercicio.cu


In [8]:
!nvcc -arch=sm_70 -std=c++14 exercicio.cu -o exercicio


In [10]:
!./exercicio < stocks-google.txt

Tempo de LEITURA (em segundos)  0.000552316
Tempo de CÓPIA (em segundos)  0.102557
Soma dos elementos: 5.29845e+06
Média dos elementos: 2104.23
Menor preço do período total: 1015.24
Maior preço do período total: 2200


In [12]:
%%writefile exercicio3.cu
#include <thrust/device_vector.h>
#include <thrust/host_vector.h>
#include <thrust/transform.h>
#include <thrust/reduce.h>
#include <iostream>
#include <fstream>   // Para manipulação de arquivos
#include <sstream>   // Para manipulação de strings
#include <chrono>    // Para medir o tempo de execução do código

// Functor para calcular a diferença entre dois valores
struct diff_functor {
    __host__ __device__
    double operator()(const double& a, const double& b) const {
        return a - b;
    }
};

int main() {
    const int n = 3520;  // Aproximadamente 3520 registros no arquivo (10 anos de dados)
    thrust::host_vector<double> host_apple(n);
    thrust::host_vector<double> host_microsoft(n);

    // Abrir o arquivo stocks2.txt
    std::ifstream file("stocks2.txt");
    if (!file) {
        std::cerr << "Erro ao abrir o arquivo stocks2.txt!" << std::endl;
        return 1;
    }

    // Ler os dados do arquivo linha por linha
    std::string line;
    int i = 0;
    while (std::getline(file, line) && i < n) {
        std::stringstream ss(line);
        std::string apple_price, microsoft_price;

        // Pega os dois valores separados por vírgula
        std::getline(ss, apple_price, ',');
        std::getline(ss, microsoft_price, ',');

        // Armazena os preços no vetor host
        host_apple[i] = std::stod(apple_price);  // Converte para double
        host_microsoft[i] = std::stod(microsoft_price);
        i++;
    }
    file.close();

    // Copiar os dados do host para a GPU
    thrust::device_vector<double> dev_apple = host_apple;
    thrust::device_vector<double> dev_microsoft = host_microsoft;
    thrust::device_vector<double> dev_diff(n);  // Vetor para armazenar as diferenças

    // Calcular a diferença ponto a ponto entre os dois vetores
    thrust::transform(dev_apple.begin(), dev_apple.end(), dev_microsoft.begin(), dev_diff.begin(), diff_functor());

    // Calcular a soma das diferenças
    double sum_diff = thrust::reduce(dev_diff.begin(), dev_diff.end(), 0.0, thrust::plus<double>());

    // Calcular a média das diferenças
    double mean_diff = sum_diff / n;

    // Exibir o resultado
    std::cout << "A diferença média entre os preços da Apple e Microsoft é: " << mean_diff << std::endl;

    return 0;
}


Writing exercicio3.cu


In [13]:
!nvcc -arch=sm_70 -std=c++14 exercicio3.cu -o exercicio3

In [14]:
!./exercicio3

A diferença média entre os preços da Apple e Microsoft é: -265.232
